# Практическая_8.Стэкинг

## Источники

- https://www.geeksforgeeks.org/machine-learning/stacking-in-machine-learning/ - исходный код для примера

- https://rasbt.github.io/mlxtend/ - документация библиотеки mlxtend

- https://sebastianraschka.com/pdf/software/mlxtend-latest.pdf - pdf документация библиотеки mlxtend 

- https://sky.pro/wiki/analytics/standardscaler-v-python-normalizaciya-dannyh-dlya-mashinnogo-obucheniya/?ysclid=mha08rzflv197255900 - методы нормализации

- 

Стекинг (stacking) в машинном обучении — метод ансамблирования, при котором конечная модель (stacked model) объединяет прогнозы из нескольких базовых моделей.

Архитектура стекинга состоит из двух частей:

1. Базовые модели (уровень-0) — первые модели, которые напрямую извлекают уроки из исходных обучающих данных. Могут быть дерево решений, логистическая регрессия, случайный лес и т. д.. Каждая модель обучается отдельно с использованием одних и тех же обучающих данных.

2. Метамодель (уровень-1) — окончательная модель, которая извлекает уроки из выходных данных базовых моделей, а не из необработанных данных. Её задача — разумно комбинировать прогнозы базовых моделей для получения окончательного прогноза. Например, простая линейная регрессия или логистическая регрессия может выступать в качестве метамодели.



Стекинг в отличии от Blending использует кросс-валидацию на всём обучающем наборе данных для получения прогнозов базовых моделей, которые затем используются для обучения мета-модели.

## Реализация Стэкинга

Импортируем нужные библиотеки

### MLxtend 

библиотека машинного обучения 

In [110]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [111]:
import pandas as pd
import matplotlib.pyplot as plt

# предобработка
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# базовые модели
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# метамодель
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier

# оценка качества моделей
from sklearn.model_selection import cross_val_score

In [112]:
df = pd.read_csv('data_8/heart.csv')    
                 
X = df.drop('target', axis = 1)
y = df['target']

df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [113]:
# познакомимся с типами признаками
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [114]:
# оценим вариативность признаков
df.nunique()

age          41
sex           2
cp            4
trestbps     49
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          4
target        2
dtype: int64

In [115]:
#  приводем данные к стандартному нормальному распределению с центром в нуле и стандартным отклонением, равным единице.
sc = StandardScaler()  

# стандартизируем только числовые признаки
var_transform = ['thalach', 'age', 'trestbps', 'oldpeak', 'chol']
X_std_scaler = X.copy()
X_std_scaler[var_transform] = sc.fit_transform(X[var_transform])   
X_std_scaler.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.952197,1,3,0.763956,-0.256334,1,0,0.015443,0,1.087338,0,0,1
1,-1.915313,1,2,-0.092738,0.072199,0,1,1.633471,0,2.122573,0,0,2
2,-1.474158,0,1,-0.092738,-0.816773,0,0,0.977514,0,0.310912,2,0,2
3,0.180175,1,1,-0.663867,-0.198357,0,1,1.239897,0,-0.206705,2,0,2
4,0.290464,0,0,-0.663867,2.082050,0,1,0.583939,1,-0.379244,2,0,2


In [116]:
# базовые модели
clf1 = RandomForestClassifier(random_state=0)
clf2 = GaussianNB()
clf3 = KNeighborsClassifier(n_neighbors=3)
meta_model = LogisticRegression(random_state=0)
# мета-модель
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=meta_model)

# список всех моделей
models_list = ['Random Forest', 'GaussianNB', 'KNN','StackingClassifier']

In [117]:
print('3-fold cross validation:\n')
for clf, label in zip([clf1, clf2, clf3, sclf],models_list):
    scores = cross_val_score(clf, X_std_scaler, y, cv=3, scoring='accuracy')
    print(f"Accuracy: {scores.mean():.2f} (+/- {scores.std():.4f}) [{label}]")

3-fold cross validation:

Accuracy: 0.82 (+/- 0.0337) [Random Forest]
Accuracy: 0.81 (+/- 0.0247) [GaussianNB]
Accuracy: 0.78 (+/- 0.0214) [KNN]
Accuracy: 0.82 (+/- 0.0337) [StackingClassifier]


### Использование вероятностей в качестве мета-признаков

В качестве альтернативы, вероятности классов классификаторов первого уровня можно использовать для обучения метаклассификатора/метамодели, установив use_probas=True. 

Если average_probas=True, вероятности классификаторов первого уровня усредняются, если average_probas=False, вероятности суммируются (рекомендуется).

In [118]:
# базовые модели
clf1 = RandomForestClassifier(random_state=0)
clf2 = GaussianNB()
clf3 = KNeighborsClassifier(n_neighbors=3)
meta_model = LogisticRegression(random_state=0)
# мета-модель
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          use_probas=True,
                          average_probas=False,
                          meta_classifier=meta_model)

print('3-fold cross validation:\n')
for clf, label in zip([clf1, clf2, clf3, sclf],models_list):
    scores = cross_val_score(clf, X_std_scaler, y, cv=3, scoring='accuracy')
    print(f"Accuracy: {scores.mean():.2f} (+/- {scores.std():.4f}) [{label}]")

3-fold cross validation:

Accuracy: 0.82 (+/- 0.0337) [Random Forest]
Accuracy: 0.81 (+/- 0.0247) [GaussianNB]
Accuracy: 0.78 (+/- 0.0214) [KNN]
Accuracy: 0.82 (+/- 0.0162) [StackingClassifier]


### Стекинг и GridSearchCV 

In [119]:
from sklearn.model_selection import GridSearchCV

# базовые модели
clf1 = RandomForestClassifier(random_state=0)
clf2 = GaussianNB()
clf3 = KNeighborsClassifier(n_neighbors=3)
meta_model = LogisticRegression(random_state=0)
# мета-модель
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          use_probas=True,
                          average_probas=False,
                          meta_classifier=meta_model)
# Параметры моделей 
params = {'kneighborsclassifier__n_neighbors': [1, 5],
 'randomforestclassifier__n_estimators': [10, 50],
 'meta_classifier__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sclf, param_grid=params, cv=5, refit=True)
grid.fit(X, y)
cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print(f"{grid.cv_results_[cv_keys[0]][r]:.3f} +/- {grid.cv_results_[cv_keys[1]][r] / 2.0:.2f} {grid.cv_results_[cv_keys[2]][r]}")

print(f'Best parameters: {grid.best_params_}')
print(f'Accuracy: {grid.best_score_:.4f}')

0.604 +/- 0.04 {'kneighborsclassifier__n_neighbors': 1, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 10}
0.598 +/- 0.04 {'kneighborsclassifier__n_neighbors': 1, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 50}
0.585 +/- 0.05 {'kneighborsclassifier__n_neighbors': 1, 'meta_classifier__C': 10.0, 'randomforestclassifier__n_estimators': 10}
0.585 +/- 0.05 {'kneighborsclassifier__n_neighbors': 1, 'meta_classifier__C': 10.0, 'randomforestclassifier__n_estimators': 50}
0.828 +/- 0.02 {'kneighborsclassifier__n_neighbors': 5, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 10}
0.822 +/- 0.02 {'kneighborsclassifier__n_neighbors': 5, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 50}
0.799 +/- 0.02 {'kneighborsclassifier__n_neighbors': 5, 'meta_classifier__C': 10.0, 'randomforestclassifier__n_estimators': 10}
0.822 +/- 0.02 {'kneighborsclassifier__n_neighbors': 5, 'meta_classifier__C': 10.0, 'randomforestclassifier_

### Класс StackingClassifier в Sklearn

sklearn.ensemble.StackingClassifier(estimators, final_estimator=None, *, cv=None, stack_method='auto', n_jobs=None, passthrough=False, verbose=0)

- estimators: Список или кортеж, содержащий базовые модели. Каждая модель должна быть экземпляром класса, поддерживающего методы fit, predict и (опционально) predict_proba.
- final_estimator: Мета-модель, которая будет использоваться для обучения на мета-данных. Если не указана, по умолчанию используется LogisticRegression.
- cv: Количество блоков кросс-валидации, используемых для генерации мета-данных. Может быть целым числом или объектом, поддерживающим кросс-валидацию.
- stack_method: Метод, используемый для объединения прогнозов базовых моделей. Может быть 'auto', 'predict_proba' или 'decision_function'.
- n_jobs: Количество параллельных процессов, используемых для обучения базовых моделей и мета-модели.
- passthrough: Если True, то исходные признаки также добавляются к мета-данным.
- verbose: Уровень детализации выводимой информации.


In [120]:
from sklearn.ensemble import StackingClassifier as StackingClassifier_sklearn
from sklearn.model_selection import GridSearchCV

# базовые модели
clf1 = RandomForestClassifier(random_state=0)
clf2 = GaussianNB()
clf3 = KNeighborsClassifier(n_neighbors=3)
meta_model = LogisticRegression(random_state=0)
# мета-модель
sclf = StackingClassifier_sklearn(estimators=[('rf', clf1), ('gnb', clf2), ('knn', clf3)], 
                          stack_method='predict_proba',
                          final_estimator=meta_model)
# Параметры моделей 
params = {'knn__n_neighbors': [1, 5],
          'rf__n_estimators': [10, 50],
          'final_estimator__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sclf, param_grid=params, cv=5, refit=True)
grid.fit(X, y)
cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print(f"{grid.cv_results_[cv_keys[0]][r]:.3f} +/- {grid.cv_results_[cv_keys[1]][r] / 2.0:.2f} {grid.cv_results_[cv_keys[2]][r]}")

print(f'Best parameters: {grid.best_params_}')
print(f'Accuracy: {grid.best_score_:.4f}')

0.822 +/- 0.02 {'final_estimator__C': 0.1, 'knn__n_neighbors': 1, 'rf__n_estimators': 10}
0.825 +/- 0.03 {'final_estimator__C': 0.1, 'knn__n_neighbors': 1, 'rf__n_estimators': 50}
0.828 +/- 0.03 {'final_estimator__C': 0.1, 'knn__n_neighbors': 5, 'rf__n_estimators': 10}
0.825 +/- 0.03 {'final_estimator__C': 0.1, 'knn__n_neighbors': 5, 'rf__n_estimators': 50}
0.828 +/- 0.03 {'final_estimator__C': 10.0, 'knn__n_neighbors': 1, 'rf__n_estimators': 10}
0.841 +/- 0.03 {'final_estimator__C': 10.0, 'knn__n_neighbors': 1, 'rf__n_estimators': 50}
0.831 +/- 0.03 {'final_estimator__C': 10.0, 'knn__n_neighbors': 5, 'rf__n_estimators': 10}
0.835 +/- 0.03 {'final_estimator__C': 10.0, 'knn__n_neighbors': 5, 'rf__n_estimators': 50}
Best parameters: {'final_estimator__C': 10.0, 'knn__n_neighbors': 1, 'rf__n_estimators': 50}
Accuracy: 0.8414


### Самостоятельная работа

Датасет 
forest_covertype.csv

1. Загрузить данные
2. Удалить выбросы и дубликаты
3. Проверить зависимость признаков.
4. Удалить признаки с высокой зависимостью.
3. Закодировать признаки с типом object
4. Разделить данные на признаки и целевую переменную
5. Разделить на подвыборки для обучения и тестовую
6. Создать стэк моделей RandomForestClassifier
7. Обучить классификатор
8. Найти значение RocAUC

### Пример реализации с нуля

In [122]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [123]:
# Создаем обучающую выборку
x_cls, y_cls = make_classification(n_samples=100, n_features=4)

# Если данных много - можно просто делить на три части
# исходный датасет поделим на тестовую и тренирочную
x_train, x_test, y_train, y_test = train_test_split(x_cls, y_cls, test_size=0.15, random_state=23)

print(f"Размер обучающей выборки: {x_train.shape}, {y_train.shape}")
print(f"Размер тестовой выборки: {x_test.shape}, {y_test.shape}")

Размер обучающей выборки: (85, 4), (85,)
Размер тестовой выборки: (15, 4), (15,)


##### Определим модели

In [125]:
# Определим внутренние модели
weak_models = [('dt', DecisionTreeClassifier()),
               ('knn', KNeighborsClassifier()),
               ('svc', SVC())]

# Мета модель или финальная модель
final_model = LogisticRegression()

##### Определим функцию обучения внутренних моделей

In [126]:
# определим функцию получения новых признаков.(обучения внутренних/базовых моделей)
def k_fold_cross_validation(inner_model, x_train, y_train, count_folds):
    predictions_clf = None
    
    # определяет размер подвыборки
    batch_size = int(len(x_train) / count_folds)

    for fold in range(count_folds):
        # если текущая подвыборка итоговая, в неё складываем все оставшиеся элементы
        if fold == (count_folds - 1):            
            batch_start = batch_size * fold 
            batch_finish = x_train.shape[0]
        else:
            batch_start = batch_size * fold
            batch_finish = batch_size * (fold + 1)

        # складываем в выборку данные текущего фолда,
        # в train складываем данные которые не вошли в тестовую выборку  [53:]
        fold_x_test = x_train[batch_start:batch_finish, :] 
        fold_x_train = x_train[[index for index in range(x_train.shape[0]) if
                                        index not in range(batch_start, batch_finish)], :]
        # аналогично для целевых переменных
        fold_y_train = y_train[
            [index for index in range(x_train.shape[0]) if index not in range(batch_start, batch_finish)]]

        # обучаем внутренний классификатор
        inner_model.fit(fold_x_train, fold_y_train)
        # получаем предсказания на данных которые не вошли в обучающую выборку 
        fold_y_pred = inner_model.predict(fold_x_test)

        # складываем в общий массив новых признаков. Размер массива в итоге [len(y_train):count_fold]
        if isinstance(predictions_clf, np.ndarray):
            predictions_clf = np.concatenate((predictions_clf, fold_y_pred))
        else:
            predictions_clf = fold_y_pred

    return predictions_clf

In [130]:
# реализуем функцию обучения и получения предсказаний на всех данных подвыборок train и test для внутренней модели [len(y_test),1]
def get_test_data_level_0(inner_model, x_train, y_train, x_test):    
    inner_model.fit(x_train, y_train)
    y_pred = inner_model.predict(x_test)

    return y_pred

# реализуем функцию для обучения мета-модели на новых метафакторах, протестируем на тестовых метафакторах
def train_meta_model(final_learner, train_meta_model_data, y_train, test_meta_model_data, y_test):
    final_learner.fit(train_meta_model_data, y_train)
    
    print(f"Train accuracy: {final_learner.score(train_meta_model_data, y_train):.4f}")
    print(f"Test accuracy: {final_learner.score(test_meta_model_data, y_test):.4f}")

In [131]:
# общая функция Стэкинга
def train_stack(weak_learners, final_learner, x_train, y_train, x_test, y_test, count_folds):
    train_meta_model_data = None
    test_meta_model_data = None

    # Start stacking
    for clf_id, clf in weak_learners:
        # Получаем метафакторы/новые признаки, как предсказанные значения на исходных данных с помощью внутреннего классификатора [len(y_train):count_fold]
        predictions_clf = k_fold_cross_validation(clf, x_train=x_train, y_train=y_train, count_folds=count_folds)

        # Получаем тестовые метафакторы как предсказания на внутренней модели, обученной на всех данных и предсказанных на всей тестовой выборке
        test_predictions_clf = get_test_data_level_0(clf, x_train=x_train, y_train=y_train, x_test=x_test)

        # объединяем признаки для обучения
        if isinstance(train_meta_model_data, np.ndarray):
            train_meta_model_data = np.vstack((train_meta_model_data, predictions_clf))
        else:
            train_meta_model_data = predictions_clf
        
        # Объединяем признаки для тестирования
        if isinstance(test_meta_model_data, np.ndarray):
            test_meta_model_data = np.vstack((test_meta_model_data, test_predictions_clf))
        else:
            test_meta_model_data = test_predictions_clf

    # Транспонируем train_meta_model
    train_meta_model_data = train_meta_model_data.T

    # Транспонируем test_meta_model
    test_meta_model_data = test_meta_model_data.T

    # Обучим мета модель
    train_meta_model(final_learner=final_learner, train_meta_model_data=train_meta_model_data, test_meta_model_data=test_meta_model_data, y_train=y_train, y_test=y_test)

In [132]:
train_stack(weak_learners=weak_models,final_learner=final_model, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test, count_folds=5)

Train accuracy: 0.8706
Test accuracy: 0.8000
